In [1]:
%matplotlib inline

In [1]:
import pennylane as qml
from pennylane import numpy as np
from qiskit_aer.noise import NoiseModel, QuantumError, pauli_error
from pennylane.optimize import NesterovMomentumOptimizer

In [2]:
# Ideal device setup
dev = qml.device("default.qubit", wires=2)

def get_angles(x):
    beta0 = 2 * np.arcsin(np.sqrt(x[1] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3] ** 2) / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(np.linalg.norm(x[2:]) / np.linalg.norm(x))
    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])

def state_preparation(a):
    qml.RY(a[0], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)
    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)

In [3]:
# testing if routine works
x = np.array([0.4746, 0.6610, 0.1, 0.1], requires_grad=False)
ang = get_angles(x)

@qml.qnode(dev)
def test(angles):
    state_preparation(angles)
    return qml.state()

state = test(ang)

print("x               : ", np.round(x, 6))
print("angles          : ", np.round(ang, 6))
print("amplitude vector: ", np.round(np.real(state), 6))

x               :  [0.4746 0.661  0.1    0.1   ]
angles          :  [ 0.344148 -0.785398  0.785398 -0.94809   0.94809 ]
amplitude vector:  [0.574623 0.800307 0.121075 0.121075]


In [4]:
def layer(layer_weights):
    for wire in range(2):
        qml.Rot(*layer_weights[wire], wires=wire)
    qml.CNOT(wires=[0, 1])

In [5]:
# Cost function
def square_loss(labels, predictions):
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

In [6]:
# Load and preprocess data
data = np.loadtxt("trainX.txt")
X = data[:, 0:2]
padding = np.ones((len(X), 2)) * 0.1
X_pad = np.c_[X, padding]
normalization = np.sqrt(np.sum(X_pad**2, -1))
X_norm = (X_pad.T / normalization).T
features = np.array([get_angles(x) for x in X_norm], requires_grad=False)

Y = np.loadtxt("trainY.txt")
Y = np.where(Y == 0, -1.0, 1.0)

In [7]:
# Apply k-means clustering
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300) # Use K-means++ for better initial centroids
# kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)
clusters = kmeans.labels_
clusters = np.where(clusters == 0, -1.0, 1.0)

In [8]:
# import matplotlib.pyplot as plt

# plt.figure()
# plt.scatter(X[:, 0][Y == 1], X[:, 1][Y == 1], c="b", marker="o", ec="k", label='Stable (Actual)')
# plt.scatter(X[:, 0][Y == -1], X[:, 1][Y == -1], c="r", marker="o", ec="k", label='Unstable (Actual)')
# plt.title("Original Data")
# plt.legend()
# plt.show()

# plt.figure()
# plt.scatter(X[:, 0][clusters == 1], X[:, 1][clusters == 1], c="b", marker="o", ec="k", label='Cluster 0')
# plt.scatter(X[:, 0][clusters == -1], X[:, 1][clusters == -1], c="r", marker="o", ec="k", label='Cluster 1')
# plt.title("K-means Clustered Data")
# plt.legend()
# plt.show()

In [9]:
# Split data into training and validation sets
np.random.seed(0)
num_data = len(clusters)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))

feats_train = features[index[:num_train]]
clusters_train = clusters[index[:num_train]]
feats_val = features[index[num_train:]]
clusters_val = clusters[index[num_train:]]

X_train = X[index[:num_train]]
X_val = X[index[num_train:]]

In [10]:
# Optimization
num_qubits = 2
num_layers = 6

weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

opt = NesterovMomentumOptimizer(0.05)
batch_size = 128

weights = weights_init
bias = bias_init

In [11]:
def setNoise(p_reset, p_meas, p_gate1):
    p_reset = float(p_reset)
    p_meas = float(p_meas)
    p_gate1 = float(p_gate1)
    
    error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
    error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
    error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_reset, "reset")
    noise_model.add_all_qubit_quantum_error(error_meas, "measure")
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x", "y", "z", "h", "s", "sdg", "t", "tdg", "rx", "ry", "rz", "p"])
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx", "cy", "cz", "swap", "rxx", "ryy", "rzz", "rzx", "cu1", "cu2", "cu3", "cp"])
    
    return noise_model

In [12]:
# testing without for loop - only on one error

# Error set definition
error_Set = [10**i for i in np.arange(-4.5, -0.9, 0.1)]

# for i_Set in range(len(error_Set)-1,-1,-1):
for i_Set in range(35,34,-1):
    # print(i_Set)
    gate_error = error_Set[i_Set]
    noise_model = setNoise(0.03, 0.04413, gate_error)
    dev_noisy = qml.device('qiskit.aer', wires=2, noise_model=noise_model)
    
    @qml.qnode(dev_noisy) 
    def noisy_circuit(weights, x):
        state_preparation(x)
        for wire in range(2):
            qml.Hadamard(wires=wire)

        for layer_weights in weights:
            layer(layer_weights)

        # Additional entangling and non-parameteric gates for expressibility 
        for i in range(2): 
             qml.CZ(wires=[i, (i+1) % 2]) 
        for i in range(2): 
             qml.S(wires=i)  
        return qml.expval(qml.PauliZ(0))

    def variational_classifier_noisy(weights, bias, x):
        return noisy_circuit(weights, x) + bias
    
    def cost_noisy(weights, bias, X, Y):
        predictions = variational_classifier_noisy(weights, bias, X.T)
        return square_loss(Y, predictions)
    
    # Train the QNN with noisy device
    for i in range(10):
        # print(i+1)
        # Update weights by one optimizer step
        batch_index = np.random.randint(0, num_train, (batch_size,))
        feats_batch = feats_train[batch_index]
        clusters_batch = clusters_train[batch_index]

        # using datetime module
        import datetime;
 
        print("before opt: ", datetime.datetime.now()) # print time before
        
        weights, bias, _, __ = opt.step(cost_noisy, weights, bias, feats_batch, clusters_batch)

        print("after opt: ", datetime.datetime.now()) # print time before

        # Compute accuracy
        predictions_train = [np.sign(variational_classifier_noisy(weights, bias, x)) for x in feats_train]
        predictions_val = [np.sign(variational_classifier_noisy(weights, bias, x)) for x in feats_val]
        acc_train = accuracy(clusters_train, predictions_train)
        acc_val = accuracy(clusters_val, predictions_val)

        print(f" Iter: {i+1} | Gate Error: {gate_error} | Cost: {cost_noisy(weights, bias, features, clusters):.7f} | Accuracy: {acc_train:.7f} | Val Acc: {acc_val:.7f}")

before opt:  2024-07-01 23:31:32.695879
after opt:  2024-07-01 23:44:39.094267
 Iter: 1 | Gate Error: 0.09999999999999713 | Cost: 1.0062315 | Accuracy: 0.5000000 | Val Acc: 0.4687500
before opt:  2024-07-01 23:45:23.988418
after opt:  2024-07-01 23:54:11.877331
 Iter: 2 | Gate Error: 0.09999999999999713 | Cost: 0.9956970 | Accuracy: 0.5000000 | Val Acc: 0.4843750
before opt:  2024-07-01 23:54:42.597874
after opt:  2024-07-02 00:05:37.824246
 Iter: 3 | Gate Error: 0.09999999999999713 | Cost: 1.0035647 | Accuracy: 0.5520833 | Val Acc: 0.5312500
before opt:  2024-07-02 00:06:09.824620
after opt:  2024-07-02 00:16:20.282652
 Iter: 4 | Gate Error: 0.09999999999999713 | Cost: 0.9900942 | Accuracy: 0.5208333 | Val Acc: 0.5000000
before opt:  2024-07-02 00:17:19.475333
after opt:  2024-07-02 00:27:50.998031
 Iter: 5 | Gate Error: 0.09999999999999713 | Cost: 0.9935464 | Accuracy: 0.5520833 | Val Acc: 0.5156250
before opt:  2024-07-02 00:28:25.330425
after opt:  2024-07-02 00:35:19.490151


KeyboardInterrupt: 

In [17]:
# FROM PENNYLANE DOCUMENTATION
# import pennylane as qml

# import qiskit
from qiskit_aer import noise

# Error probabilities
prob_1 = 0.001  # 1-qubit gate
prob_2 = 0.01   # 2-qubit gate

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

# Create a PennyLane device
dev = qml.device('qiskit.aer', wires=2, noise_model=noise_model)

# Create a PennyLane quantum node run on the device
@qml.qnode(dev)
def circuit(x, y, z):
    qml.RZ(z, wires=[0])
    qml.RY(y, wires=[0])
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=1))

# Result of noisy simulator
print(circuit(0.2, 0.1, 0.3))

0.95703125
